In [5]:
import numpy as np
from numpy.linalg import pinv, solve
from collections import defaultdict


In [6]:
def load_data(path):
    data = np.loadtxt(path)
    y_raw = data[:, 0].astype(int)
    X = data[:, 1:]

    #  map: 1 to -1, 3 to +1
    y = np.where(y_raw == 1, -1.0, 1.0)
    return X, y


In [7]:
def laplacian(W):
    D = np.diag(W.sum(axis=1))
    return D - W

In [8]:
def create_knn_graph(X, k=3):
    m = X.shape[0]

    norms = np.sum(X**2, axis=1)
    D = norms[:, None] + norms[None, :] - 2 * X @ X.T
    np.fill_diagonal(D, np.inf)

    W = np.zeros((m, m))

    for i in range(m):
        nn = np.argpartition(D[i], k)[:k]
        W[i, nn] = 1
        W[nn, i] = 1 

    return W


In [9]:
def LI(L, y, L_idx):
    m = len(y)
    mask = np.ones(m, dtype=bool)
    mask[L_idx] = False
    U_idx = np.where(mask)[0]

    u = np.zeros(m)
    u[L_idx] = y[L_idx]

    if len(U_idx) == 0:
        return u

    L_UU = L[np.ix_(U_idx, U_idx)]
    L_UL = L[np.ix_(U_idx, L_idx)]

    try:
        u[U_idx] = solve(L_UU, -L_UL @ y[L_idx])
    except:
        u[U_idx] = pinv(L_UU) @ (-L_UL @ y[L_idx])

    return u


In [10]:
def LKI(L, y, L_idx):
    G = pinv(L)
    K = G[np.ix_(L_idx, L_idx)]
    yL = y[L_idx]

    alpha = pinv(K) @ yL
    v = G[:, L_idx] @ alpha

    return v


In [11]:
def error_rate(y_true, y_pred, L_idx):
    mask = np.ones(len(y_true), dtype=bool)
    mask[L_idx] = False
    return np.mean(y_pred[mask] != y_true[mask])


In [12]:
def sample_labels(y, l):
    idx_neg = np.where(y == -1)[0]
    idx_pos = np.where(y == +1)[0]

    L_neg = np.random.choice(idx_neg, l, replace=False)
    L_pos = np.random.choice(idx_pos, l, replace=False)

    return np.concatenate([L_neg, L_pos])


In [13]:
def run_experiment(file):
    X, y = load_data(file)
    W = create_knn_graph(X)
    L = laplacian(W)

    results_LI = defaultdict(list)
    results_LKI = defaultdict(list)

    for l in [1, 2, 4, 8, 16]:
        for _ in range(20):
            L_idx = sample_labels(y, l)
            # Laplacian interpolation
            u_li = LI(L, y, L_idx)
            yhat_li = np.sign(u_li)
            yhat_li[yhat_li == 0] = 1
            results_LI[l].append(error_rate(y, yhat_li, L_idx))

            # Laplacian kernel interpolatio
            v_lki = LKI(L, y, L_idx)
            yhat_lki = np.sign(v_lki)
            yhat_lki[yhat_lki == 0] = 1
            results_LKI[l].append(error_rate(y, yhat_lki, L_idx))

    return results_LI, results_LKI


In [14]:
files = [
    "dtrain13_50.dat",
    "dtrain13_100.dat",
    "dtrain13_200.dat",
    "dtrain13_400.dat"
]

for f in files:
    print("\nDataset:", f)
    LI_res, LKI_res = run_experiment(f)

    print("LI:")
    for l in LI_res:
        print(f"  ℓ={l}: {np.mean(LI_res[l]):.2f} ± {np.std(LI_res[l]):.2f}")

    print("LKI:")
    for l in LKI_res:
        print(f"  ℓ={l}: {np.mean(LKI_res[l]):.2f} ± {np.std(LKI_res[l]):.2f}")



Dataset: dtrain13_50.dat
LI:
  ℓ=1: 0.24 ± 0.15
  ℓ=2: 0.14 ± 0.09
  ℓ=4: 0.08 ± 0.04
  ℓ=8: 0.05 ± 0.02
  ℓ=16: 0.04 ± 0.02
LKI:
  ℓ=1: 0.13 ± 0.11
  ℓ=2: 0.07 ± 0.04
  ℓ=4: 0.06 ± 0.03
  ℓ=8: 0.04 ± 0.01
  ℓ=16: 0.04 ± 0.02

Dataset: dtrain13_100.dat
LI:
  ℓ=1: 0.05 ± 0.02
  ℓ=2: 0.04 ± 0.01
  ℓ=4: 0.04 ± 0.01
  ℓ=8: 0.05 ± 0.02
  ℓ=16: 0.03 ± 0.01
LKI:
  ℓ=1: 0.05 ± 0.01
  ℓ=2: 0.04 ± 0.01
  ℓ=4: 0.04 ± 0.01
  ℓ=8: 0.04 ± 0.02
  ℓ=16: 0.03 ± 0.01

Dataset: dtrain13_200.dat
LI:
  ℓ=1: 0.13 ± 0.13
  ℓ=2: 0.04 ± 0.06
  ℓ=4: 0.03 ± 0.03
  ℓ=8: 0.02 ± 0.01
  ℓ=16: 0.02 ± 0.01
LKI:
  ℓ=1: 0.03 ± 0.02
  ℓ=2: 0.02 ± 0.01
  ℓ=4: 0.02 ± 0.01
  ℓ=8: 0.02 ± 0.01
  ℓ=16: 0.02 ± 0.00

Dataset: dtrain13_400.dat
LI:
  ℓ=1: 0.07 ± 0.12
  ℓ=2: 0.04 ± 0.06
  ℓ=4: 0.01 ± 0.00
  ℓ=8: 0.01 ± 0.00
  ℓ=16: 0.01 ± 0.00
LKI:
  ℓ=1: 0.01 ± 0.00
  ℓ=2: 0.01 ± 0.00
  ℓ=4: 0.01 ± 0.00
  ℓ=8: 0.01 ± 0.00
  ℓ=16: 0.01 ± 0.00
